In [239]:
# Инициализируем Nornir
from nornir import InitNornir
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
from nornir_jinja2.plugins.tasks import template_file
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result

In [240]:
# Загружаем профиль
nr = InitNornir(config_file="./config.yml",
    runner={
        "plugin": "threaded",
        "options": {
            "num_workers": 50,
        },
    },

    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "./inventory/hosts.yml",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml"
        },
    },
)

In [241]:
# Указываем путь к папке с шаблонами
templates_path = "./templates/"

In [242]:
sw = nr.filter(type="l2",)

for host in sw.inventory.hosts.keys():
    print(nr.inventory.hosts[host])
   # print(nr.inventory.hosts[host].data["interface_list"])
    nr.inventory.hosts[host].username = "sa"
    nr.inventory.hosts[host].password = "Qwerty11"
    nr.inventory.hosts[host].connection_options['netmiko'].extras['secret'] = "Qwerty11"
    

SW-1
SW-2


In [243]:
#result = sw.run(netmiko_send_command, command_string="show ip int brief")
#print_result(result)

In [244]:
def config_switches(task:Task) -> Result:
    # ------------------------------- Configure the interface -------------------------------------------------------------------------------------------------------------------- ----------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_interface_template = task.run(
        name='2 шаг. 1: Получаем шаблон конфигурации интерфейсов',  # mission name
        task=template_file,   # The purpose of the task is to get the configuration template, fixed value
        template='cisco_ios_interface.template',
        path=templates_path
    )

    # Introducing in a specific configuration, configuring the device, pay attention to ".split ('\ n')" Convert configuration to a list
    task.run(
        task=netmiko_send_config,  # The purpose of the task is to complete the configuration, fixed value through Netmiko.
        name='2 шаг. 2: Настройка интерфейсов',
        config_commands=ios_interface_template.result.split('\n'), # Convert template information to a list, then configure it via NetMiko
        cmd_verify=True
    )
    
    # --------------------------------- Configure administrator information ------------ ----------------
    # Получаем шаблон и отправляем в конфигурацию
    ios_user_template = task.run(
        name='1 шаг. 1: Получаем шаблон конфигурации пользователей',
        task=template_file,
        template='1-cisco_ios_user.template',
        path=templates_path
    )
    # Introducing in a specific configuration, configuring the device, pay attention to ".split ('\ n')" Convert configuration to a list
    task.run(
        task=netmiko_send_config,
        name='1 шаг. 2: Настройка пользователей',
        config_commands=ios_user_template.result.split('\n'),
        cmd_verify=True
    )

    ios_ntp_template = task.run(
        name='3 шаг. 1: Получаем шаблон конфигурации ntp',
        task=template_file,
        template='cisco_ios_ntp.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='3 шаг. 2: Настройка ntp',
        config_commands=ios_ntp_template.result.split('\n'),
        cmd_verify=True
    )
    #-------------
    ios_dns_template = task.run(
        name='4 шаг. 1: Получаем шаблон конфигурации dns',
        task=template_file,
        template='cisco_ios_dns.template',
        path=templates_path
    )
    task.run(
        task=netmiko_send_config,
        name='4 шаг. 2: Настройка dns',
        config_commands=ios_dns_template.result.split('\n'),
        cmd_verify=True
    )
    #-------------
    return Result(
        host=task.host,
        result=f"{task.host} доступен"
    )

In [245]:
run_result = sw.run(task=config_switches,name='Настройка коммутаторов')
print_result(run_result)

Настройка коммутаторов**********************************************************
* SW-1 ** changed : True *******************************************************
vvvv Настройка коммутаторов ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
SW-1 доступен
---- 2 шаг. 1: Получаем шаблон конфигурации интерфейсов ** changed : False ----- INFO
    interface vlan1
          description management
          ip address 10.30.253.11 255.255.255.0
          no shutdown

---- 2 шаг. 2: Настройка интерфейсов ** changed : True ------------------------- INFO
configure terminal
Enter configuration commands, one per line.  End with CNTL/Z.
Switch(config)#    interface vlan1
Switch(config-if)#          description management
Switch(config-if)#          ip address 10.30.253.11 255.255.255.0
Switch(config-if)#          no shutdown
Switch(config-if)#
Switch(config-if)#end
Switch#
---- 1 шаг. 1: Получаем шаблон конфигурации пользователей ** changed : False --- INFO
ip domain name plgn.com
enable sec